In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

project_path = '/content/drive/My Drive/Code Documentation Project/Issue Classification/'

##compare stand alone

In [ ]:
import pandas as pd

# Initialize an empty list to store DataFrames
dfs = []

for fold in range(1, 11):
    #test_df = pd.read_csv(project_path + '/Ensemble Data/Literature/fold' + str(fold) + '.csv')
    test_df = pd.read_csv(project_path + '/Ensemble Data/OpenJ9/fold' + str(fold) + '.csv')
    dfs.append(test_df)

# Concatenate all DataFrames in the list into one DataFrame
issue_df = pd.concat(dfs, ignore_index=True)

print(len(issue_df))
print(issue_df.head())
#issue_df.info()
#issue_df[['Code','RF','LGBM','BERT','RoBERTa']]

In [ ]:
import pandas as pd

# Example DataFrame setup
data = {
    'Code': [1, 2, 3, 1, 2],  # Original labels
    'LGBM_TFIDF': [1, 1, 3, 2, 1],  # Predictions from LightGBM
    'RF': [1, 2, 2, 1, 3],  # Predictions from Random Forest
    'BertCW': [1, 3, 3, 1, 2],  # Predictions from BERT
    'RobertaCW': [2, 2, 3, 1, 2]  # Predictions from RoBERTa
}

#issue_df = pd.DataFrame(data)
models=['RF','LGBM','BERT','RoBERTa']
total_mis_all=[]
def get_correction_stat(model):
  misclassified = issue_df[issue_df[model] != issue_df['Code']]
  total_mis=len(misclassified)
  total_mis_all.append(total_mis)

  correct=[]
  for m in models:
    if(m==model):
      correct.append(0)
      continue

    corrected_by = (misclassified['Code'] == misclassified[m]).sum()/ total_mis
    correct.append(round(corrected_by, 2))

  return correct


corr_df = pd.DataFrame(index=models, columns=models)
for model in models:
    corr_df.loc[model] = get_correction_stat(model)
print(corr_df)


In [ ]:
def get_at_least_correction_stat(model):
    misclassified = issue_df[issue_df[model] != issue_df['Code']]
    total_mis = len(misclassified)

    corrected_by_at_least_one = 0
    for idx, row in misclassified.iterrows():
        other_models = [m for m in models if m != model]
        if row['Code'] in [row[m] for m in other_models]:
            corrected_by_at_least_one += 1

    return corrected_by_at_least_one/total_mis

# Create a DataFrame to store the correction stats
corr_by_at_least_one_df = pd.DataFrame(index=models, columns=['Misclassifications Corrected by at Least One Model'])

# Calculate and store the correction stats for each model
for model in models:
    corr_by_at_least_one_df.loc[model, 'Misclassifications Corrected by at Least One Model'] = get_at_least_correction_stat(model)

# Print the DataFrame
corr_by_at_least_one_df = corr_by_at_least_one_df.applymap(lambda x: round(x, 2))
corr_df['>=1']=corr_by_at_least_one_df
corr_df['wrong']=total_mis_all

corr_df.to_csv(project_path + '/classification report/Misclassification/OpenJ9CorrectionIndidiualModels.csv',index=False)
corr_df

#compare ensemble

In [ ]:
import numpy as np
# Function to calculate corrections
def calculate_corrections(df, original, model_pred, correctors):
    correction_counts = []
    for label in np.unique(df[original]):
        # Misclassified by the model
        misclassified = df[(df[original] == label) & (df[model_pred] != label)]
        misclassified_label_len=len(misclassified)
        corrections = {}
        for corrector in correctors:
            # Corrected by each ensemble
            corrected = misclassified[(misclassified[original] == misclassified[corrector])]
            corrections[f"Corrected by {corrector}"] = len(corrected)/misclassified_label_len
        correction_counts.append([label] + list(corrections.values()))
    return pd.DataFrame(correction_counts, columns=['Label'] + list(corrections.keys()))

# Calculate corrections
correction_df = calculate_corrections(issue_df, 'Code', 'LGBM', ['Ens_LGBM', 'Ens_RF','Ens_BERT','Ens_RoBERTa'])

correction_df['Label'] = correction_df['Label'].replace(label_mapping)

#correction_df.to_csv(project_path + '/classification report/Misclassification/LiteratureCorrectionEnsemble.csv',index=False)
correction_df.to_csv(project_path + '/classification report/Misclassification/OpenJ9CorrectionEnsemble.csv',index=False)
correction_df
